<a href="https://colab.research.google.com/github/Tikquuss/grokking_beyong_l2_norm/blob/main/compressed_sensing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ...

In [ ]:
# !git clone https://github.com/Tikquuss/grokking_beyong_l2_norm
# %cd grokking_beyong_l2_norm
# # #! ls
# ! pip install -r requirements.txt
LOG_DIR="/content/LOGS"

In [ ]:
import os
os.makedirs(LOG_DIR, exist_ok=True)

In [ ]:
%matplotlib inline

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
from plotters.utils import get_twin_axis, FIGSIZE, LINEWIDTH, FIGSIZE_SMALL, FIGSIZE_LARGE, FIGSIZE_MEDIUM, FONTSIZE, LABEL_FONTSIZE, TICK_LABEL_FONTSIZE, MARKERSIZE
from plotters.img_show import custom_imshow

from matplotlib.lines import Line2D
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
print(default_colors)

In [ ]:
# Utils
from sparse_recovery.utils import sample_iterations_for_plotting, select_log_space
from sparse_recovery.utils import plot_t1_t2, find_memorization_generalization_steps, find_stable_step_final_value
# Basis $\Phi$
from sparse_recovery.compressed_sensing import create_Fourier_basis, create_orthonormal_basis, create_normal_basis
# Signal $a^*$ and $b^*=\Phi a^*$, Measures $X$, and Coherence
from sparse_recovery.compressed_sensing import create_signal, create_noise_from_scratch, create_noise_from_mean_norm_signal, get_measures, calculate_coherence
### Optimization
from sparse_recovery.compressed_sensing import solve_compressed_sensing_l1, subgradient_descent, deep_subgradient_descent, get_gradient

# Coherence : Figure 19(a)

In [ ]:
# Example usage
n = 100  # Dimension of Phi
seed = None
# Generate Phi
#Phi = create_Fourier_basis(n) # Fourier basis
Phi = create_orthonormal_basis(n, scaler=None, seed=seed) # Q of QR decomposition
#Phi = create_normal_basis(n, scaler=None, seed=seed, normalized=False) # Random Normal
#Phi = np.eye(n)

N_max = n# int(0.9 * n)
all_N = list(range(1, N_max, 1)) # Number of measurements, N < n
all_tau = np.linspace(0, 1, 1*10+1)  # Range of tau values to test
all_tau = np.linspace(0, 1, 2*10+1) # Range of tau values to test

coherences = [ [] for _ in range(len(all_N)) ]
for i, N in tqdm(enumerate(all_N), total=len(all_N)) :
    for j, tau in enumerate(all_tau):
        coherence_value = 0
        n_trials = 10
        for k in range(n_trials):
            M, X = get_measures(N, Phi, tau=tau, variance=1/N, seed=None)
            #N_1 = int(tau*N)
            #M = M[:max(N_1, 1), :]
            #M = M[min(N_1, N-1):, :]
            coherence_value += calculate_coherence(M.T, Phi)
        coherences[i].append(coherence_value / n_trials)

In [ ]:
rows, cols = 1, 1
figsize=FIGSIZE
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
color_indices = np.linspace(0, 1, len(all_N))
colors = plt.cm.viridis(color_indices)
for i, N in enumerate(all_N):
    ax.plot(all_tau, coherences[i], label=f'N={N}', color=colors[i])
ax.set_xlabel('$\\tau$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('coherence', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
#ax.set_xscale('log')
#ax.set_yscale('log')
#ax.grid()
#ax.legend()
# Create a color bar for the sparsity levels `s`
sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_N), vmax=max(all_N)))
sm.set_array([])  # We only need the colormap here, no actual data
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label('N', fontsize=LABEL_FONTSIZE)

#plt.savefig(f"{LOG_DIR}/coherence_tau"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

# Convex Programming : Figures 19(b) and 20

In [ ]:
n = 100 # Dimension of the signal
seed = None
signal_distribution="normal"
#Phi = create_Fourier_basis(n) # Fourier basis
Phi = create_orthonormal_basis(n, scaler=None, seed=seed)
#Phi = np.random.randn(n, n) * (1/np.sqrt(n))
Phi = np.eye(n)

#SNR=10**8  # signal to noise ratio (np.inf for no noise)
SNR=np.inf

#all_s =  [1, 2]
all_s = [s for s in list(range(1, n+10, 10)) if s <= n] # Sparsity level <= n
#all_s = [s for s in list(range(1, n+10, 5)) if s <= n] # Sparsity level <= n

#all_N =  [15]
all_N = list(range(1, n+11, 10)) # Number of measurements N
#all_N = list(range(1, n+11, 1))

all_tau = np.arange(10+1)/10  # Range of tau values to test
all_tau = [0.0, 0.5, 1.0]
#all_tau = [0.0]

errors = {}
min_N_for_s = {}

for iii, tau in enumerate(all_tau):
    print(f"tau = {tau}, {iii+1}/{len(all_tau)}")
    errors[tau] = [ [] for _ in range(len(all_s)) ]
    min_N_for_s[tau] = [None for _ in range(len(all_s))]
    for i, s in tqdm(enumerate(all_s), total=len(all_s)):
        a_star, b_star = create_signal(n, s, signal_distribution, Phi, scaler=None, seed=None)
        for j, N in enumerate(all_N) :
            error_mean = 0
            n_trials = 4 # 10
            for k in range(n_trials):
                # Construct a random Gaussian measurement matrix X
                M, X = get_measures(N, Phi, tau=tau, variance=1/N, seed=None)
                xi = create_noise_from_scratch(N, SNR, n, s, signal_distribution, Phi, scaler=None, seed=None)
                # Generate measurements y
                y_star = X @ a_star + xi # M @ b_star + xi
                # Solve the l1-minimization problem (P1) to recover a
                a = solve_compressed_sensing_l1(X, y_star, EPSILON=1e-8)
                # Evaluate the recovery
                recovery_error = np.linalg.norm(a - a_star, 2) / np.linalg.norm(a_star, 2)
                error_mean += recovery_error

            error_mean /= n_trials
            errors[tau][i].append(error_mean)
            if error_mean < 1e-6 and min_N_for_s[tau][i] is None:
                min_N_for_s[tau][i] = N

## Figure 20

In [ ]:
from matplotlib.colors import LogNorm

rows, cols = len(all_tau), 3
figsize=FIGSIZE
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

for iii, tau in enumerate(all_tau):
    ax = fig.add_subplot(rows, cols, 1+3*iii)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    color_indices = np.linspace(0, 1, len(all_s))
    colors = plt.cm.viridis(color_indices)
    for i, s in enumerate(all_s):
        ax.plot(all_N, errors[tau][i], label=f's={s}', color=colors[i])
    ax.set_xlabel('Number of measurements (N)', fontsize=LABEL_FONTSIZE)
    ax.set_ylabel(f'Error ($\\tau$ = {round(tau, 3)})', fontsize=LABEL_FONTSIZE)
    ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
    ax.set_yscale('log')
    #ax.grid()
    #ax.legend()
    # Create a color bar for the sparsity levels `s`
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_s), vmax=max(all_s)))
    sm.set_array([])  # We only need the colormap here, no actual data
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label('Sparsity level (s)', fontsize=LABEL_FONTSIZE)

    ax = fig.add_subplot(rows, cols, 2+3*iii)
    img_data = np.array(errors[tau]) # (M, N) = (len(all_s), len(all_N))
    img = custom_imshow(
        img_data, ax=ax, fig=fig, add_text=False,
        hide_ticks_and_labels=False, xticklabels=all_N, yticklabels=all_s,
        filter_step_xticks=10, filter_step_yticks=3, log_x=False, log_y=False, base=10,
        rotation_x=90, rotation_y=0,
        x_label="Number of measurements (N)",  y_label="Sparsity level (s)",
        # Use LogNorm to apply a logarithmic scale
        colormesh_kwarg={"shading":'auto', "cmap":'viridis', 'norm':LogNorm(vmin=img_data.min(), vmax=img_data.max())},
        imshow_kwarg={},
        colorbar=True, colorbar_label='Error',
        label_fontsize=LABEL_FONTSIZE,
        ticklabel_fontsize=TICK_LABEL_FONTSIZE,
        show=False, fileName=None, dpf=None
    )

    ax = fig.add_subplot(rows, cols, 3+3*iii)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    ax.plot(all_s, min_N_for_s[tau], label="true")
    #ax.plot(all_s, [s * np.log(n) for s in all_s], label="predict")
    ax.set_xlabel('Sparsity level (s)', fontsize=LABEL_FONTSIZE)
    ax.set_ylabel('$N_{min}$ for recovery', fontsize=LABEL_FONTSIZE)
    ax.yaxis.tick_right()  # Move y-axis ticks to the right
    ax.yaxis.set_label_position("right")  # Move y-axis label to the right
    ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
    #ax.legend()
    #ax.grid()

# Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

# #plt.savefig(f"{LOG_DIR}/compressed_sensing_convex_programming_all"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')
# plt.savefig(f"{LOG_DIR}/compressed_sensing_convex_programming"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

## Figure 19(b)

In [ ]:
rows, cols = 1, 1
figsize=FIGSIZE
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
color_indices = np.linspace(0, 1, len(all_tau))
colors = plt.cm.viridis(color_indices)
for i, tau in enumerate(all_tau):
    ax.plot(all_s, min_N_for_s[tau], label=f'$\\tau$={round(tau, 3)}', color=colors[i])

ax.set_xlabel('Sparsity level (s)', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$N_{min}$ for recovery', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
# Create a color bar
sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_tau), vmax=max(all_tau)))
sm.set_array([])  # We only need the colormap here, no actual data
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label('$\\tau$', fontsize=LABEL_FONTSIZE)
# Set the ticks to correspond to the values in `all_tau`
cbar.set_ticks(all_tau)  # Sets tick positions based on `all_tau`
cbar.set_ticklabels([str(tau) for tau in all_tau])  # Sets tick labels to match `all_tau`

#plt.savefig(f"{LOG_DIR}/compressed_sensing_convex_programming_N_min"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

# Gradient Descent : Subgradient & Projected & Proximal (ISTA) Gradient Descent

In [ ]:
LABELS_PLOTS = {
    'train_errors' : '$||X a^{(t)} - y^*||_2 \ \ / \ \ ||y^*||_2$',
    'errors' : '$||a^{(t)} - a^*||_2 \ \ / \ \ ||a^*||_2$',
}

## Gradients Norms, $a^{(t)}$ norm : Figures 3 and 25

In [ ]:
n = 100 # Dimension of the signal
seed = 1
signal_distribution="normal"
#Phi = create_Fourier_basis(n) # Fourier basis
Phi = create_orthonormal_basis(n, scaler=None, seed=seed)
#Phi = np.random.randn(n, n) * (1/np.sqrt(n))
Phi = np.eye(n)

s=5
N=30
SNR=10**8 # signal to noise ratio (np.inf for no noise)
#SNR=np.inf

a_star, b_star = create_signal(n, s, signal_distribution, Phi, scaler=None, seed=seed)
M, X = get_measures(N, Phi, tau=0.0, variance=1/N, seed=seed)
xi = create_noise_from_scratch(N, SNR, n, s, signal_distribution, Phi, scaler=None, seed=seed)
y_star = X @ a_star + xi

alpha = 1e-1
beta_1 = 1e-5
beta_2 = 0.0
init_scale = 1e-6

method = "subgradient"
# method = "proj_subgradient"
# method = "ISTA"

max_iter=10**6*1+1

# T = max_iter/eval_step
# a~(n), errors~(T), train_errors~(T), all_a~(T, n), iterations~(T), a_LS~(n)
a, errors, train_errors, all_a, iterations, a_LS = subgradient_descent(
    X, y_star, a_star, method=method, learning_rate=alpha, beta_2=beta_2, beta_1=beta_1, max_iter=max_iter, init_scale=init_scale, tol=1e-6)

# fileName = f"compressed_sensing_test"
# exp_dir = f"{LOG_DIR}/{fileName}"
# os.makedirs(exp_dir, exist_ok=True)
# np.save(f"{exp_dir}/a_star.npy", a_star)
# np.save(f"{exp_dir}/a.npy", a)
# np.save(f"{exp_dir}/errors.npy", errors)
# np.save(f"{exp_dir}/train_errors.npy", train_errors)
# np.save(f"{exp_dir}/all_a.npy", all_a)
# np.save(f"{exp_dir}/iterations.npy", iterations)
# np.save(f"{exp_dir}/a_LS.npy", a_LS)

In [ ]:
zero_indexes = np.where(a_star == 0)[0]
non_zero_indexes = np.where(a_star != 0)[0]
a_recovered = all_a[-1]
#print(a_star.round(4), a_recovered.round(1))

print(a_star[non_zero_indexes].round(4), a_recovered[non_zero_indexes].round(4))
print(a_star[zero_indexes].round(4), a_recovered[zero_indexes].round(4))

### Figure 3

In [ ]:
is_complex = np.iscomplexobj(X)
_, _, norms_grad_ratio = get_gradient(X, y_star, all_a, beta_1, is_complex=is_complex, ord=2)

plt.plot(iterations, norms_grad_ratio)
plt.xscale("log")
#plt.yscale("log")

In [ ]:
threshold = 1e-3
t_1, t_2 = find_memorization_generalization_steps(train_errors, test_errors=errors, train_steps=iterations, train_threshold=threshold, test_threshold=threshold)
print(t_1, t_2)

In [ ]:
rows, cols = 1, 3
rows, cols = 3, 1
figsize=(8, 4)
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

color_1, color_2 = default_colors[0], default_colors[1]

log_x=True
log_y=True

all_steps = iterations + []
if log_x : all_steps = np.array(all_steps) + 1

##############################################################################
##############################################################################

ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

ax.plot(all_steps, errors, '--', color=color_1, linewidth=LINEWIDTH)
ax.plot(all_steps, train_errors, '-', color=color_1, linewidth=LINEWIDTH)

############### plot Delta_t

plot_t1_t2(ax, t_1, t_2, log_x, log_y, plot_Delta=True)

###############

if cols==3:ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_1)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)

if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

## Norm gradient
ax1.plot(all_steps, norms_grad_ratio, "-", color=color_2, linewidth=LINEWIDTH)

#ax1.set_ylabel('$||\\beta h(a)||_2 \ / \ ||G(a)||_2 $', fontsize=LABEL_FONTSIZE, color=color_2)
ax1.set_ylabel('Gradients Ratio', fontsize=LABEL_FONTSIZE, color=color_2)
ax1.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

legend_elements = [
    Line2D([0], [0], color=color_1, linestyle='-', label=LABELS_PLOTS['train_errors']),
    Line2D([0], [0], color=color_1, linestyle='--', label=LABELS_PLOTS['errors']),
    Line2D([0], [0], color=color_2, linestyle='-', label='$||\\beta H(a^{(t)})||_2 \ \ / \ \ ||G(a^{(t)})||_2$')
    ]
ax1.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8, loc='best')


##############################################################################
##############################################################################

ax = fig.add_subplot(rows, cols, 2)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

ax.plot(all_steps, errors, '--', color=color_1, linewidth=LINEWIDTH)
ax.plot(all_steps, train_errors, '-', color=color_1, linewidth=LINEWIDTH)
if cols==3:ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
if rows==3:ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_1)

############### plot Delta_t

plot_t1_t2(ax, t_1, t_2, log_x, log_y, plot_Delta=True)

###############

if cols==2:ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
if rows==2:ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_1)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)

if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

## Norm a
A = np.array(all_a) # (n_steps, n)
y = np.linalg.norm(A, ord=1, axis=1)
ax1.plot(all_steps, y, "-", color=color_2, linewidth=LINEWIDTH)

norm_a_star = np.linalg.norm(a_star, ord=1)
ax1.axhline(y=norm_a_star, color=color_2, linestyle='--', alpha=0.8, linewidth=LINEWIDTH)

ax1.set_ylabel('$||a||_1$', fontsize=LABEL_FONTSIZE, color=color_2)
ax1.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

legend_elements = [
    #Line2D([0], [0], color=color_1, linestyle='-', label=LABELS_PLOTS['train_errors']),
    #Line2D([0], [0], color=color_1, linestyle='--', label=LABELS_PLOTS['errors']),
    Line2D([0], [0], color=color_2, linestyle='-', label='$||a^{(t)}||_1$'),
    Line2D([0], [0], color=color_2, linestyle='--', label='$||a^*||_1$')
    ]
ax1.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8, loc='best')

#if log_x : ax1.set_xscale('log')
if log_y : ax1.set_yscale('log')


##############################################################################
##############################################################################

ax = fig.add_subplot(rows, cols, 3)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
#_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

color_indices = np.linspace(0, 1, n)
colors = plt.cm.viridis(color_indices)
for i in range(n):
    ax.plot(np.array(iterations)+1, A[:,i], '-', color=colors[i])

############### plot Delta_t

plot_t1_t2(ax, t_1, t_2, log_x, log_y, plot_Delta=False)

###############

if rows==3:ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Components of $a^{(t)}$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE)

if log_x : ax.set_xscale('log')
# if log_y : ax.set_yscale('symlog')

# sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=1, vmax=n))
# sm.set_array([])  # We only need the colormap here, no actual data
# cbar = plt.colorbar(sm, ax=ax)
# cbar.set_label(f'Dimensions (n)', fontsize=LABEL_FONTSIZE)

ax.axhline(y=0, color='gray', linestyle='--', linewidth=0.8, alpha=0.8)
#ax.axhline(y=-0.01, color='gray', linestyle='--', linewidth=0.8, alpha=0.8)

##############################################################################
##############################################################################

# # Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

#plt.savefig(f"{LOG_DIR}/compressed_sensing_gradient_norm_b_{method}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

### Figure 25

In [ ]:
A = np.array(all_a) # (n_steps, n)
threshold_ajusted = -np.inf
threshold_ajusted = 1e-4
A = (np.abs(A)>=threshold_ajusted) * A # ajust (keep only the values > threshold_ajusted and set the rest to 0)

skip_step = 1#0**1 * 1
img_data = A[::skip_step,:]
print(img_data.shape)

In [ ]:
rows, cols = 1, 1
figsize=FIGSIZE
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(rows, cols, 1)

##
img = custom_imshow(
    img_data, ax=ax, fig=fig, add_text=False,
    hide_ticks_and_labels=False, xticklabels=np.arange(1, img_data.shape[1]+1), yticklabels=all_steps[::skip_step],
    filter_step_xticks=5, filter_step_yticks=10**0, log_x=False, log_y=True, base=10,
    rotation_x=90, rotation_y=0,
    x_label="Dimensions (n)",  y_label="Steps (t)",
    # Use LogNorm to apply a logarithmic scale
    colormesh_kwarg={"shading":'auto', "cmap":'viridis'}, # 'norm':LogNorm(vmin=img_data.min(), vmax=img_data.max())
    imshow_kwarg={},
    colorbar=True, colorbar_label='',
    label_fontsize=LABEL_FONTSIZE,
    ticklabel_fontsize=TICK_LABEL_FONTSIZE,
    show=False, fileName=None, dpf=None
)


## Overlay plot for b_star on top
ax_a_star = fig.add_axes([ax.get_position().x0, ax.get_position().y1 + 0.05, ax.get_position().width-0.045, 0.05])
cax_a_star = ax_a_star.imshow(a_star[None,:] , aspect="auto", cmap="viridis") # norm=LogNorm(vmin=img_data.min(), vmax=img_data.max())
ax_a_star.set_xticks([])
ax_a_star.set_yticks([])
ax_a_star.set_ylabel("$a*$", fontsize=LABEL_FONTSIZE, rotation=90)
ax_a_star.yaxis.set_label_position("right")

##
# plt.savefig(f"{LOG_DIR}/compressed_sensing_gradient_components_{method}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')
# plt.savefig(f"{LOG_DIR}/compressed_sensing_gradient_components_{method}"  + '.png', dpi=300, bbox_inches='tight')

plt.show()

## Large init & $\ell_2$ : Figures 7 and 16 (Grokking Without Understanding)

In [ ]:
n = 100 # Dimension of the signal
seed = 1
signal_distribution="normal"
#Phi = create_Fourier_basis(n) # Fourier basis
Phi = create_orthonormal_basis(n, scaler=None, seed=seed)
#Phi = np.random.randn(n, n) * (1/np.sqrt(n))
Phi = np.eye(n)

s=5
N=30
SNR=10**8 # signal to noise ratio (np.inf for no noise)
#SNR=np.inf

a_star, b_star = create_signal(n, s, signal_distribution, Phi, scaler=None, seed=seed)
M, X = get_measures(N, Phi, tau=0.0, variance=1/N, seed=seed)
xi = create_noise_from_scratch(N, SNR, n, s, signal_distribution, Phi, scaler=None, seed=seed)
y_star = X @ a_star + xi

alpha = 1e-1
beta_1 = 0
beta_2 = 1e-5
init_scale = 10.0

method = "subgradient"
#method = "proj_subgradient"
# method = "ISTA"

max_iter=10**8*1+1

a, errors, train_errors, all_a, iterations, a_LS = subgradient_descent(
    X, y_star, a_star, method=method, learning_rate=alpha, beta_2=beta_2, beta_1=beta_1, max_iter=max_iter, init_scale=init_scale, tol=1e-6)

In [ ]:
zero_indexes = np.where(a_star == 0)[0]
non_zero_indexes = np.where(a_star != 0)[0]
a_recovered = all_a[-1]
#print(a_star.round(4), a_recovered.round(1))

print(a_star[non_zero_indexes].round(4), a_recovered[non_zero_indexes].round(4))
print(a_star[zero_indexes].round(4), a_recovered[zero_indexes].round(4))

In [ ]:
threshold = 1e-3
t_1, t_2 = find_memorization_generalization_steps(train_errors, test_errors=errors, train_steps=iterations, train_threshold=threshold, test_threshold=threshold)
print(t_1, t_2)

### Figure 16

In [ ]:
rows, cols = 1, 2
#rows, cols = 2, 1
figsize=(8, 4)
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

color_1, color_2 = default_colors[0], default_colors[1]

log_x=True

log_y=False
#log_y=True

all_steps = iterations + []
if log_x : all_steps = np.array(all_steps) + 1

##############################################################################
##############################################################################

ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

ax.plot(all_steps, errors, '--', color=color_1, linewidth=LINEWIDTH)
ax.plot(all_steps, train_errors, '-', color=color_1, linewidth=LINEWIDTH)

############### plot Delta_t

plot_t1_t2(ax, t_1, t_2, log_x, log_y, plot_Delta=True)

###############

if cols==2:ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_1)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)

if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

## Norm b (l2)
A = np.array(all_a) # (n_steps, n)
y = np.linalg.norm(A, ord=1, axis=1)
ax1.plot(all_steps, y, "-", color=color_2, linewidth=LINEWIDTH)

norm_b_star = np.linalg.norm(b_star, ord=2)
ax1.axhline(y=norm_b_star, color=color_2, linestyle='--', alpha=0.8, linewidth=LINEWIDTH)

#ax1.set_ylabel('$||a||_1$', fontsize=LABEL_FONTSIZE, color=color_2)
ax1.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

legend_elements = [
    Line2D([0], [0], color=color_1, linestyle='-', label=LABELS_PLOTS['train_errors']),
    Line2D([0], [0], color=color_1, linestyle='--', label=LABELS_PLOTS['errors']),
    #Line2D([0], [0], color=color_2, linestyle='-', label='$||a^{(t)}||_1$'),
    #Line2D([0], [0], color=color_2, linestyle='--', label='$||a^*||_1$')
    ]
ax1.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

#if log_x : ax1.set_xscale('log')
if log_y : ax1.set_yscale('log')

##############################################################################
##############################################################################
#log_y=False
log_y=True

ax = fig.add_subplot(rows, cols, 2)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

ax.plot(all_steps, errors, '--', color=color_1, linewidth=LINEWIDTH)
ax.plot(all_steps, train_errors, '-', color=color_1, linewidth=LINEWIDTH)

############### plot Delta_t

plot_t1_t2(ax, t_1, t_2, log_x, log_y, plot_Delta=False)

###############

ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
#if rows==2:ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_1)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)

if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

## Norm a
A = np.array(all_a) # (n_steps, n)
y = np.linalg.norm(A, ord=1, axis=1)
ax1.plot(all_steps, y, "-", color=color_2, linewidth=LINEWIDTH)

norm_a_star = np.linalg.norm(a_star, ord=1)
ax1.axhline(y=norm_a_star, color=color_2, linestyle='--', alpha=0.8, linewidth=LINEWIDTH)

ax1.set_ylabel('$||a||_1$', fontsize=LABEL_FONTSIZE, color=color_2)
ax1.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

legend_elements = [
    #Line2D([0], [0], color=color_1, linestyle='-', label=LABELS_PLOTS['train_errors']),
    #Line2D([0], [0], color=color_1, linestyle='--', label=LABELS_PLOTS['errors']),
    Line2D([0], [0], color=color_2, linestyle='-', label='$||a^{(t)}||_1$'),
    Line2D([0], [0], color=color_2, linestyle='--', label='$||a^*||_1$')
    ]
ax1.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

#if log_x : ax1.set_xscale('log')
if log_y : ax1.set_yscale('log')

##############################################################################
##############################################################################

# # Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

#plt.savefig(f"{LOG_DIR}/compressed_sensing_error_large_init_log_vs_nonlog"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

### Figure 7

In [ ]:
rows, cols = 1, 3
#rows, cols = 3, 1
figsize=(8, 4)
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

color_1, color_2 = default_colors[0], default_colors[1]

log_x=True

log_y=False
#log_y=True

all_steps = iterations + []
if log_x : all_steps = np.array(all_steps) + 1

##############################################################################
##############################################################################

ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

ax.plot(all_steps, errors, '--', color=color_1, linewidth=LINEWIDTH)
ax.plot(all_steps, train_errors, '-', color=color_1, linewidth=LINEWIDTH)

############### plot Delta_t

plot_t1_t2(ax, t_1, t_2, log_x, log_y, plot_Delta=True)

###############

if cols==2:ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_1)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)

if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

## Norm b (l2)
A = np.array(all_a) # (n_steps, n)
y = np.linalg.norm(A, ord=2, axis=1)
ax1.plot(all_steps, y, "-", color=color_2, linewidth=LINEWIDTH)

norm_b_star = np.linalg.norm(b_star, ord=2)
ax1.axhline(y=norm_b_star, color=color_2, linestyle='--', alpha=0.8, linewidth=LINEWIDTH)

ax1.set_ylabel('$||a||_2$', fontsize=LABEL_FONTSIZE, color=color_2)
ax1.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

legend_elements = [
    Line2D([0], [0], color=color_1, linestyle='-', label=LABELS_PLOTS['train_errors']),
    Line2D([0], [0], color=color_1, linestyle='--', label=LABELS_PLOTS['errors']),
    Line2D([0], [0], color=color_2, linestyle='-', label='$||a^{(t)}||_2$'),
    Line2D([0], [0], color=color_2, linestyle='--', label='$||a^*||_2$')
    ]
ax1.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

#if log_x : ax1.set_xscale('log')
if log_y : ax1.set_yscale('log')

##############################################################################
##############################################################################

ax = fig.add_subplot(rows, cols, 2)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

ax.plot(all_steps, errors, '--', color=color_1, linewidth=LINEWIDTH)
ax.plot(all_steps, train_errors, '-', color=color_1, linewidth=LINEWIDTH)

############### plot Delta_t

plot_t1_t2(ax, t_1, t_2, log_x, log_y, plot_Delta=False)

###############

ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
if rows==2:ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_1)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)

if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

## Norm a
A = np.array(all_a) # (n_steps, n)
y = np.linalg.norm(A, ord=1, axis=1)
ax1.plot(all_steps, y, "-", color=color_2, linewidth=LINEWIDTH)

norm_a_star = np.linalg.norm(a_star, ord=1)
ax1.axhline(y=norm_a_star, color=color_2, linestyle='--', alpha=0.8, linewidth=LINEWIDTH)

ax1.set_ylabel('$||a||_1$', fontsize=LABEL_FONTSIZE, color=color_2)
ax1.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

legend_elements = [
    Line2D([0], [0], color=color_1, linestyle='-', label=LABELS_PLOTS['train_errors']),
    Line2D([0], [0], color=color_1, linestyle='--', label=LABELS_PLOTS['errors']),
    Line2D([0], [0], color=color_2, linestyle='-', label='$||a^{(t)}||_1$'),
    Line2D([0], [0], color=color_2, linestyle='--', label='$||a^*||_1$')
    ]
ax1.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

#if log_x : ax1.set_xscale('log')
if log_y : ax1.set_yscale('log')

##############################################################################
##############################################################################

##############################################################################
##############################################################################

ax = fig.add_subplot(rows, cols, 3)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
#_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

color_indices = np.linspace(0, 1, n)
colors = plt.cm.viridis(color_indices)
for i in range(n):
    ax.plot(np.array(iterations)+1, A[:,i], '-', color=colors[i])

############### plot Delta_t

plot_t1_t2(ax, t_1, t_2, log_x, log_y, plot_Delta=True)

###############

if rows==3 or rows==1: ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Components of $a^{(t)}$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE)
ax.yaxis.set_label_position("right")

if log_x : ax.set_xscale('log')
# if log_y : ax.set_yscale('symlog')

# sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=1, vmax=n))
# sm.set_array([])  # We only need the colormap here, no actual data
# cbar = plt.colorbar(sm, ax=ax)
# cbar.set_label(f'Dimensions (n)', fontsize=LABEL_FONTSIZE)

ax.axhline(y=0, color='gray', linestyle='--', linewidth=0.8, alpha=0.8)
#ax.axhline(y=-0.01, color='gray', linestyle='--', linewidth=0.8, alpha=0.8)

##############################################################################
##############################################################################

# # Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

#plt.savefig(f"{LOG_DIR}/compressed_sensing_gradient_norm_b_{method}_large_init{'_log' if log_y else ''}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

Figure 7

In [ ]:
rows, cols = 1, 2
#rows, cols = 2, 1
figsize=(8, 4)
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

color_1, color_2 = default_colors[0], default_colors[1]

log_x=True

log_y=False
log_y=True

all_steps = iterations + []
if log_x : all_steps = np.array(all_steps) + 1

##############################################################################
##############################################################################

ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

ax.plot(all_steps, errors, '--', color=color_1, linewidth=LINEWIDTH)
ax.plot(all_steps, train_errors, '-', color=color_1, linewidth=LINEWIDTH)

############### plot Delta_t

plot_t1_t2(ax, t_1, t_2, log_x, log_y, plot_Delta=True)

###############

if cols==2:ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_1)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)

if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

## Norm b (l2)
A = np.array(all_a) # (n_steps, n)
y = np.linalg.norm(A, ord=2, axis=1)
ax1.plot(all_steps, y, "-", color=color_2, linewidth=LINEWIDTH)

norm_b_star = np.linalg.norm(b_star, ord=2)
ax1.axhline(y=norm_b_star, color=color_2, linestyle='--', alpha=0.8, linewidth=LINEWIDTH)

ax1.set_ylabel('$||a||_2$', fontsize=LABEL_FONTSIZE, color=color_2)
ax1.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

legend_elements = [
    Line2D([0], [0], color=color_1, linestyle='-', label=LABELS_PLOTS['train_errors']),
    Line2D([0], [0], color=color_1, linestyle='--', label=LABELS_PLOTS['errors']),
    Line2D([0], [0], color=color_2, linestyle='-', label='$||a^{(t)}||_2$'),
    Line2D([0], [0], color=color_2, linestyle='--', label='$||a^*||_2$')
    ]
ax1.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

#if log_x : ax1.set_xscale('log')
if log_y : ax1.set_yscale('log')

##############################################################################
##############################################################################

# ax = fig.add_subplot(rows, cols, 2)
# _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
# _, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

# ax.plot(all_steps, errors, '--', color=color_1, linewidth=LINEWIDTH)
# ax.plot(all_steps, train_errors, '-', color=color_1, linewidth=LINEWIDTH)

# ############### plot Delta_t

# plot_t1_t2(ax, t_1, t_2, log_x, log_y, plot_Delta=False)

# ###############

# ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
# if rows==2:ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_1)
# ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
# ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)

# if log_x : ax.set_xscale('log')
# if log_y : ax.set_yscale('log')

# ## Norm a
# A = np.array(all_a) # (n_steps, n)
# y = np.linalg.norm(A, ord=1, axis=1)
# ax1.plot(all_steps, y, "-", color=color_2, linewidth=LINEWIDTH)

# norm_a_star = np.linalg.norm(a_star, ord=1)
# ax1.axhline(y=norm_a_star, color=color_2, linestyle='--', alpha=0.8, linewidth=LINEWIDTH)

# ax1.set_ylabel('$||a||_1$', fontsize=LABEL_FONTSIZE, color=color_2)
# ax1.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

# legend_elements = [
#     Line2D([0], [0], color=color_1, linestyle='-', label=LABELS_PLOTS['train_errors']),
#     Line2D([0], [0], color=color_1, linestyle='--', label=LABELS_PLOTS['errors']),
#     Line2D([0], [0], color=color_2, linestyle='-', label='$||a^{(t)}||_1$'),
#     Line2D([0], [0], color=color_2, linestyle='--', label='$||a^*||_1$')
#     ]
# ax1.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

# #if log_x : ax1.set_xscale('log')
# if log_y : ax1.set_yscale('log')

##############################################################################
##############################################################################

##############################################################################
##############################################################################

ax = fig.add_subplot(rows, cols, 2)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
#_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

color_indices = np.linspace(0, 1, n)
colors = plt.cm.viridis(color_indices)
for i in range(n):
    ax.plot(np.array(iterations)+1, A[:,i], '-', color=colors[i])

############### plot Delta_t

plot_t1_t2(ax, t_1, t_2, log_x, log_y, plot_Delta=True)

###############

#if rows==3:
ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Components of $a^{(t)}$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE)
ax.yaxis.set_label_position("right")

if log_x : ax.set_xscale('log')
# if log_y : ax.set_yscale('symlog')

# sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=1, vmax=n))
# sm.set_array([])  # We only need the colormap here, no actual data
# cbar = plt.colorbar(sm, ax=ax)
# cbar.set_label(f'Dimensions (n)', fontsize=LABEL_FONTSIZE)

ax.axhline(y=0, color='gray', linestyle='--', linewidth=0.8, alpha=0.8)
#ax.axhline(y=-0.01, color='gray', linestyle='--', linewidth=0.8, alpha=0.8)

##############################################################################
##############################################################################

# # Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

#plt.savefig(f"{LOG_DIR}/compressed_sensing_gradient_norm_b_{method}_large_init{'_log' if log_y else ''}_for_main_section"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

In [ ]:
A = np.array(all_a) # (n_steps, n)
threshold_ajusted = -np.inf
threshold_ajusted = 1e-4
A = (np.abs(A)>=threshold_ajusted) * A # ajust (keep only the values > threshold_ajusted and set the rest to 0)

skip_step = 10**1 * 1
#skip_step = 10**2 * 1
img_data = A[::skip_step,:]
print(img_data.shape)

In [ ]:
rows, cols = 1, 1
figsize=FIGSIZE
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(rows, cols, 1)

##
img = custom_imshow(
    img_data, ax=ax, fig=fig, add_text=False,
    hide_ticks_and_labels=False, xticklabels=np.arange(1, img_data.shape[1]+1), yticklabels=all_steps[::skip_step],
    filter_step_xticks=5, filter_step_yticks=10**0, log_x=False, log_y=True, base=10,
    rotation_x=90, rotation_y=0,
    x_label="Dimensions (n)",  y_label="Steps (t)",
    # Use LogNorm to apply a logarithmic scale
    colormesh_kwarg={"shading":'auto', "cmap":'viridis'}, # 'norm':LogNorm(vmin=img_data.min(), vmax=img_data.max())
    imshow_kwarg={},
    colorbar=True, colorbar_label='',
    label_fontsize=LABEL_FONTSIZE,
    ticklabel_fontsize=TICK_LABEL_FONTSIZE,
    show=False, fileName=None, dpf=None
)


## Overlay plot for a_star on top
ax_a_star = fig.add_axes([ax.get_position().x0, ax.get_position().y1 + 0.05, ax.get_position().width-0.045, 0.05])
cax_a_star = ax_a_star.imshow(a_star[None,:] , aspect="auto", cmap="viridis") # norm=LogNorm(vmin=img_data.min(), vmax=img_data.max())
ax_a_star.set_xticks([])
ax_a_star.set_yticks([])
ax_a_star.set_ylabel("$a*$", fontsize=LABEL_FONTSIZE, rotation=90)
ax_a_star.yaxis.set_label_position("right")

##
# plt.savefig(f"{LOG_DIR}/compressed_sensing_gradient_components_{method}_large_init"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')
# plt.savefig(f"{LOG_DIR}/compressed_sensing_gradient_components_{method}_large_init"  + '.png', dpi=300, bbox_inches='tight')

plt.show()

## Scaling wrt N & Convergence to least square : Figure 24

In [ ]:
n = 100  # Dimension of the signal
seed = 1
signal_distribution="normal"
#Phi = create_Fourier_basis(n) # Fourier basis
Phi = create_orthonormal_basis(n, scaler=None, seed=seed)
#Phi = np.random.randn(n, n) * (1/np.sqrt(n))
Phi = np.eye(n)

s=5
a_star, b_star = create_signal(n, s, signal_distribution, Phi, scaler=None, seed=seed)

alpha = 1e-1
beta_1 = 1e-5
beta_2 = 0.0
init_scale = 1e-6

method = "subgradient"
#method = "proj_subgradient"
# method = "ISTA"

max_iter=10**6*1+1

all_errors = {}
all_train_errors = {}
all_iterations = {}
all_a = {}
all_a_LS = {}

all_X = {}

all_N =  [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
#all_N =  [30, 70]
SNR=10**8 # signal to noise ratio (np.inf for no noise)
#SNR=np.inf

for i, N in enumerate(all_N):
    print(f"N = {N}, {(i+1)}/{len(all_N)}")

    M, X = get_measures(N, Phi, tau=0.0, variance=1/N, seed=seed+i) #
    xi = create_noise_from_scratch(N, SNR, n, s, signal_distribution, Phi, scaler=None, seed=seed+i)
    y_star = X @ a_star + xi

    a, errors, train_errors, all_a_tmp, iterations, a_LS = subgradient_descent(
        X, y_star, a_star, method=method, learning_rate=alpha, beta_2=beta_2, beta_1=beta_1, max_iter=max_iter, init_scale=init_scale, tol=1e-6)

    all_errors[N] = errors
    all_train_errors[N] = train_errors
    all_iterations[N] = iterations
    all_a[N] = all_a_tmp
    all_a_LS[N] = a_LS

    all_X[N] = X

In [ ]:
# ############### plot Delta_t

all_t1 = []
all_t1_approx = []
all_t2 = []
all_t2_approx = []
all_Delta_t = []
all_Delta_t_approx = []

all_train_errors_N = []
all_test_errors_N = []

threshold = 1 * 1e-3
#threshold = 4 * 1e-4
test_threshold = 4 * 1e-4
for i, N in enumerate(all_N):

    all_steps = all_iterations[N]

    t_1, t_2 = find_memorization_generalization_steps(all_train_errors[N], test_errors=all_errors[N], train_steps=all_steps, train_threshold=threshold, test_threshold=test_threshold)

    t_2, t_2_index = find_stable_step_final_value(all_steps, all_errors[N], K=3, tolerance_fraction=0.05, M=2)


    all_t1.append(t_1)
    t_2 = t_2 if t_2 is not None else all_steps[-1]
    all_t2.append(t_2)
    Delta_t = t_2 - t_1
    all_Delta_t.append(Delta_t)

    ##############################
    X = all_X[N]
    #rho = np.linalg.norm(np.eye(n) - alpha * (X.T @ X + beta_2 * np.eye(n)), ord=2) # 2-norm (largest sing. value)

    _, Sigma, _ = np.linalg.svd(X.T @ X, full_matrices=True) # (n, n), (n,), (n, n)
    r = np.linalg.matrix_rank(X.T @ X)
    rho = np.max(1 - alpha * (Sigma[:r] + beta_2 ))
    #print(Sigma[:r])
    #rho = max(rho, 1-alpha*beta_2)
    #print(rho)

    a1 = all_a[N][0]
    a_LS = all_a_LS[N]
    #a_LS = all_a[N][all_steps.index(t_1)]
    t_1_approx = - np.log(1+(1-rho) * np.linalg.norm(a1 - a_LS, ord=2) / (alpha*beta_1)) / np.log(rho+1e-8)
    all_t1_approx.append(t_1_approx)

    ##############################

    Delta_t_approx = np.linalg.norm(a_LS-a_star, ord=np.inf)/(alpha*beta_1)
    t_2_approx = t_1_approx + Delta_t_approx
    all_t2_approx.append(t_2_approx)
    #print(Delta_t)
    all_Delta_t_approx.append(Delta_t_approx)
    ##############################

    all_train_errors_N.append(all_train_errors[N][all_steps.index(t_1)])
    all_test_errors_N.append(all_errors[N][-1])

    ##############################

    print(t_1, t_1_approx)
    print(t_2, t_2_approx)
    print(Delta_t, Delta_t_approx)

    print("====")

In [ ]:
rows, cols = 1, 1
figsize=(8, 4)
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=True
log_y=True

ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
#_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

color_indices = np.linspace(0, 1, len(all_N)+1*0)
colors = plt.cm.viridis(color_indices)

for i, N in enumerate(all_N):
    all_steps = all_iterations[N]
    if log_x : all_steps = np.array(all_steps) + 1
    # Find the index of a specific step
    max_T = None
    max_T = np.abs(all_steps - 10**6).argmin()
    ax.plot(all_steps[:max_T], all_errors[N][:max_T], '--', color=colors[i], linewidth=LINEWIDTH)
    ax.plot(all_steps[:max_T], all_train_errors[N][:max_T], '-', label=f'$N$={N}', color=colors[i], linewidth=LINEWIDTH)

    t=None
    #t=all_t1[i]
    #t=all_t2[i]
    if t is not None :
        ax.axvline(x=t, ymin=0.01, ymax=1., color=colors[i], linestyle='--', lw=1.)
        ax.plot([t, t], [0, 0], 'o', color='b')

ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE)
########### Color bar
sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_N), vmax=max(all_N)))
sm.set_array([])  # We only need the colormap here, no actual data
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label(f'$N$', fontsize=LABEL_FONTSIZE)
# Set the ticks to correspond to the values in `all_N`
cbar.set_ticks(all_N)  # Sets tick positions based on `all_N`
cbar.set_ticklabels([str(n) for n in all_N])  # Sets tick labels to match `all_N`


if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

legend_elements = [
    Line2D([0], [0], color='k', linestyle='-', label=LABELS_PLOTS['train_errors']),
    Line2D([0], [0], color='k', linestyle='--', label=LABELS_PLOTS['errors']),
    ]
ax.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

##############################################################################
##############################################################################

# # Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_N_{method}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

In [ ]:
#data_to_plot = list(zip([all_t1], [all_t1_approx], ['t_1']))
data_to_plot = list(zip([all_t1, all_t2], [all_t1_approx, all_t2_approx], ['t_1', 't_2']))
#data_to_plot = list(zip([all_t1, all_t2, all_Delta_t], [all_t1_approx, all_t2_approx, all_Delta_t_approx], ['t_1', 't_2', '\Delta t']))

rows, cols = 1, len(data_to_plot)
figsize=FIGSIZE # (8, 4)
figsize=FIGSIZE_MEDIUM
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=False
log_y=True

color_1, color_2 = default_colors[0], default_colors[1]

for k, (t_experiment, t_approx, label) in enumerate(data_to_plot) :

    ax = fig.add_subplot(rows, cols, k+1)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    _, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

    ax.plot(all_N, t_experiment, '-', label=f'${label}$ (experiments)', color=color_1, linewidth=LINEWIDTH)
    ax.plot(all_N, t_approx, '--', label=f'$O({label})$ (theory)', color=color_1, linewidth=LINEWIDTH)

    ax.set_xlabel('N', fontsize=LABEL_FONTSIZE)
    if k==0 : ax.set_ylabel('t', fontsize=LABEL_FONTSIZE)
    ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
    ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)

    #ax.legend(fontsize=LABEL_FONTSIZE*0.8)

    if log_x : ax.set_xscale('log')
    if log_y : ax.set_yscale('log')

    ###################

    ax1.plot(all_N, all_train_errors_N, '-', label='$||\\tilde{X} b^{(t_1)} - y^*||_2 \ \ / \ \ ||y^*||_2$', color=color_2, linewidth=LINEWIDTH)
    ax1.plot(all_N, all_test_errors_N, '--', label='$||b^{(\infty)} - b^*||_2 \ \ / \ \ ||b^*||_2$', color=color_2, linewidth=LINEWIDTH)

    #ax1.set_xlabel('N', fontsize=LABEL_FONTSIZE)
    if k==cols-1 : ax1.set_ylabel('Error', fontsize=LABEL_FONTSIZE)
    #ax1.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
    ax1.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

    #if log_x : ax1.set_xscale('log')
    if log_y : ax1.set_yscale('log')

    legend_elements = [
        Line2D([0], [0], color=color_1, linestyle='-', label=f'${label}$ (experiments)'),
        Line2D([0], [0], color=color_1, linestyle='--', label=f'$O({label})$ (theory)'),
        Line2D([0], [0], color=color_2, linestyle='-', label='$||X a^{(t_1)} - y^*||_2 \ \ / \ \ ||y^*||_2$'),
        Line2D([0], [0], color=color_2, linestyle='--', label='$||a^{(\infty)} - a^*||_2 \ \ / \ \ ||a^*||_2$')
        ]
    ax1.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

# # Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_N_vs_times_{method}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

### Figure 24

In [ ]:
# threshold = 1e-3
# t_1, t_2 = find_memorization_generalization_steps(all_train_errors[N], test_errors = all_errors[N], train_steps=iterations, train_threshold=threshold, test_threshold=threshold)
# print(t_1, t_2)

# T=N_times_to_plot+2
# all_times = sample_iterations_for_plotting(train_steps=iterations, t1=t_1, t2=t_2, T=T, include_first=False, include_last=False, sampling_strategy='log', base=10)
# log_indices = [iterations.index(t) for t in all_times]
# print(all_times, log_indices)

In [ ]:
N = all_N[0]
iterations = all_iterations[N]

N_times_to_plot=5
markers = ['^', 'o', 's', "8", '*', 'x', 'd']  # Unique markers for each time
all_times, log_indices = select_log_space(iterations, N_times_to_plot+1, base=10)
all_times, log_indices = all_times[1:], log_indices[1:]

print(all_times, log_indices)

In [ ]:
zero_indexes = np.where(a_star == 0)[0]
non_zero_indexes = np.where(a_star != 0)[0]

In [ ]:
rows, cols = 1, 1
figsize=(6*2, 4*2)
figsize=FIGSIZE_SMALL
figsize=FIGSIZE_MEDIUM
figsize=FIGSIZE_LARGE
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)

color_indices = np.linspace(0, 1, len(all_N)+1*0)
colors = plt.cm.viridis(color_indices)

for i, N in enumerate(all_N):
    #if N<n: continue
    #if N>=n: continue

    all_a_tmp = all_a[N]
    A = np.array(all_a_tmp) # (n_steps, n)
    u_t = np.linalg.norm(A[:,non_zero_indexes], axis=1)
    v_t = np.linalg.norm(A[:,zero_indexes], axis=1)

    #ax.plot(u_t, v_t, color=colors[i], label=f'N={N}')
    #ax.scatter(u_t, v_t, c=iterations, cmap='viridis', s=1, marker='.')
    ax.scatter(u_t, v_t, color=colors[i], s=4, marker='.')

    for idx, (time_idx, t) in enumerate(zip(log_indices, all_times)):
        try :
            u_t_i, v_t_i = u_t[time_idx], v_t[time_idx]
            ax.plot(u_t_i, v_t_i, markers[idx], markersize=8, color=colors[i])
            ax.text(
                u_t_i, v_t_i+0.01/4,
                #f"$t_{idx}={t}$",
                f"$t_{idx}$",
                fontsize=15, color=colors[i],
                ha='right' if idx % 2 == 0 else 'left'
            )
        except :
            pass

    ## hat_a
    hat_a = all_a_LS[N]
    u_hat_a = np.linalg.norm(hat_a[non_zero_indexes])
    v_hat_a = np.linalg.norm(hat_a[zero_indexes])
    ax.plot(u_hat_a, v_hat_a, 'X', markersize=8, color="red")
    # ax.text(
    #     u_hat_a, v_hat_a+0.01/4,
    #     #f"$t_{idx}={t}$",
    #     f"$N={N}$",
    #     fontsize=15, color=colors[i],
    # )

    ## a^*
    u_a_star = np.linalg.norm(a_star[non_zero_indexes])
    v_a_star = np.linalg.norm(a_star[zero_indexes])
    ax.plot([u_a_star], [v_a_star], "o", markersize=8, color="red")

# Legend for times
legend_elements = [
    plt.Line2D([0], [0], marker="X", color='w', label=f"$\\hat{{a}}$ (least square)", markerfacecolor='red', markersize=8),
    plt.Line2D([0], [0], marker="o", color='w', label=f"$a^*$ (optimum)", markerfacecolor='red', markersize=8)
]

legend_elements.extend([
    plt.Line2D([0], [0], marker=markers[idx], color='w', label=f"$t_{idx}={all_times[idx]}$", markerfacecolor='k', markersize=8)
    for idx in range(N_times_to_plot)
])
ax.legend(handles=legend_elements, loc='best', fontsize=LABEL_FONTSIZE)

ax.set_xlabel('$u(t)$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel("$v(t)$", fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)

# Create a color bar for N
sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_N), vmax=max(all_N)))
sm.set_array([])  # We only need the colormap here, no actual data
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label(f'$N$', fontsize=LABEL_FONTSIZE)
# Set the ticks to correspond to the values in `all_N`
cbar.set_ticks(all_N)  # Sets tick positions based on `all_N`
cbar.set_ticklabels([str(n) for n in all_N])  # Sets tick labels to match `all_N`


# Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

# plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_N_b_landscape_{method}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')
# plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_N_b_landscape_{method}"  + '.png', dpi=300, bbox_inches='tight')

plt.show()

## Scaling wrt N and s : Figures 12 (projected subgradient), 13 (ISTA), 27 and 28 (subgradient)

In [ ]:
n = 100 # Dimension of the signal
seed = 1
signal_distribution="normal"
#Phi = create_Fourier_basis(n) # Fourier basis
Phi = create_orthonormal_basis(n, scaler=None, seed=seed)
#Phi = np.random.randn(n, n) * (1/np.sqrt(n))
Phi = np.eye(n)

all_s =  [2]
all_s =  [1, 5, 10, 15]
# # all_s =  [s for s in list(range(1, n+10, 5)) if s <= n] # Sparsity level <= n

all_N =  [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
# all_N =  [30, 70]
SNR=10**8 # signal to noise ratio (np.inf for no noise)
SNR=np.inf

alpha = 1e-1
beta_1 = 1e-5
beta_2 = 0.0
init_scale = 1e-6

method = "subgradient"
method = "proj_subgradient"
method = "ISTA"

max_iter=10**6*1+1

all_errors = {}
all_train_errors = {}
all_iterations = {}
all_a = {}
all_a_star = {}
all_a_LS = {}

all_X = {}

fileName = f"compressed_scaling_N_s_{method}"

for i, s in enumerate(all_s) :
    #print(f"s = {s}, {(i+1)}/{len(all_s)}")

    a_star, b_star = create_signal(n, s, signal_distribution, Phi, scaler=None, seed=seed)

    all_errors[s] = {}
    all_train_errors[s] = {}
    all_iterations[s] = {}
    all_a[s] = {}
    all_a_LS[s] = {}
    all_a_star[s] = a_star
    all_X[s] = {}

    for j, N in enumerate(all_N):
        print(f"s = {s}, N = {N}, {(i+1)}/{len(all_s)}, {(j+1)}/{len(all_N)}")

        M, X = get_measures(N, Phi, tau=0.0, variance=1/N, seed=seed)
        xi = create_noise_from_scratch(N, SNR, n, s, signal_distribution, Phi, scaler=None, seed=seed)
        y_star = X @ a_star + xi

        a, errors, train_errors, all_a_tmp, iterations, a_LS = subgradient_descent(
            X, y_star, a_star, method=method, learning_rate=alpha, beta_2=beta_2, beta_1=beta_1, max_iter=max_iter, init_scale=init_scale, tol=1e-6)

        all_errors[s][N] = errors
        all_train_errors[s][N] = train_errors
        all_iterations[s][N] = iterations
        all_a[s][N] = all_a_tmp
        all_a_LS[s][N] = a_LS
        all_X[s][N] = X

        exp_dir = f"{LOG_DIR}/{fileName}/s={s}_N={N}"
        os.makedirs(exp_dir, exist_ok=True)
        np.save(f"{exp_dir}/a_star.npy", a_star)
        np.save(f"{exp_dir}/a.npy", a)
        np.save(f"{exp_dir}/errors.npy", errors)
        np.save(f"{exp_dir}/train_errors.npy", train_errors)
        np.save(f"{exp_dir}/all_a.npy", all_a_tmp)
        np.save(f"{exp_dir}/iterations.npy", iterations)
        np.save(f"{exp_dir}/a_LS.npy", a_LS)

### Figures 12 (projected subgradient), 13 (ISTA) and 27 (subgradient)

In [ ]:
L=len(all_s)
cols = min(2, L)
rows = L // cols + 1 * (L % cols != 0)

figsize=(8, 4)
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=True
log_y=True
#log_y=False

for i, s in enumerate(all_s) :

    ax = fig.add_subplot(rows, cols, i+1)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    #_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

    ax.set_title(f"s={s}", fontsize=LABEL_FONTSIZE)

    color_indices = np.linspace(0, 1, len(all_N)+1*0)
    colors = plt.cm.viridis(color_indices)

    for j, N in enumerate(all_N):
        all_steps = all_iterations[s][N]
        if log_x : all_steps = np.array(all_steps)+1
        ax.plot(all_steps, all_errors[s][N], '--', color=colors[j], linewidth=LINEWIDTH)
        ax.plot(all_steps, all_train_errors[s][N], '-', label=f'$N$={N}', color=colors[j], linewidth=LINEWIDTH)

        t=None
        #t=all_t1[s][j]
        if t is not None :
            ax.axvline(x=t, ymin=0.01, ymax=1., color=colors[j], linestyle='--', lw=1.)
            ax.plot([t, t], [0, 0], 'o', color='b')

    ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
    ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE)
    ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
    ########### Color bar
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_N), vmax=max(all_N)))
    sm.set_array([])  # We only need the colormap here, no actual data
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label(f'$N$', fontsize=LABEL_FONTSIZE)
    # Set the ticks to correspond to the values in `all_N`
    cbar.set_ticks(all_N)  # Sets tick positions based on `all_N`
    cbar.set_ticklabels([str(n) for n in all_N])  # Sets tick labels to match `all_N`

    if log_x : ax.set_xscale('log')
    if log_y : ax.set_yscale('log')

    legend_elements = [
        Line2D([0], [0], color='k', linestyle='-', label=LABELS_PLOTS['train_errors']),
        Line2D([0], [0], color='k', linestyle='--', label=LABELS_PLOTS['errors']),
        ]
    ax.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

##############################################################################
##############################################################################

# # Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

##
#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_N_and_s_{method}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

In [ ]:
# ############### plot Delta_t

threshold = 1 * 1e-3
#threshold = 4 * 1e-4
all_threshold = [ 1e-2, 1e-3, 1e-3, 1e-3]
test_threshold = 4 * 1e-4

all_t1 = {}
all_t1_approx = {}
all_t2 = {}
all_t2_approx = {}
all_Delta_t = {}
all_Delta_t_approx = {}

all_train_errors_s_N = {}
all_test_errors_s_N = {}

for i, s in enumerate(all_s) :

    threshold = all_threshold[i]

    all_t1[s] = []
    all_t1_approx[s] = []
    all_t2[s] = []
    all_t2_approx[s] = []
    all_Delta_t[s] = []
    all_Delta_t_approx[s] = []
    all_train_errors_s_N[s] = []
    all_test_errors_s_N[s] = []

    for j, N in enumerate(all_N):

        all_steps = all_iterations[s][N]
        t_1, t_2 = find_memorization_generalization_steps(
            all_train_errors[s][N], test_errors=all_errors[s][N], train_steps=all_steps , train_threshold=threshold, test_threshold=test_threshold)

        t_2, t_2_index = find_stable_step_final_value(all_steps, all_errors[s][N], K=3, tolerance_fraction=0.05, M=2)

        all_t1[s].append(t_1)
        t_2 = t_2 if t_2 is not None else all_steps[-1]
        all_t2[s].append(t_2)
        Delta_t = t_2 - t_1
        all_Delta_t[s].append(Delta_t)

        ##############################
        X = all_X[s][N]
        #rho = np.linalg.norm(np.eye(n) - alpha * (X_tilde.T @ X_tilde + beta_2 * np.eye(n)), ord=2) # 2-norm (largest sing. value)

        _, Sigma, _ = np.linalg.svd(X.T @ X, full_matrices=True) # (n, n), (n,), (n, n)
        r = np.linalg.matrix_rank(X.T @ X)
        rho = np.max(1 - alpha * (Sigma[:r] + beta_2 ))
        #rho = max(rho, 1-alpha*beta_2)
        #print(rho)

        #rho = np.linalg.norm( np.eye(n) - alpha * (X.T @ X + beta_2) , ord='fro')

        a1 = all_a[s][N][0]
        a_LS = all_a_LS[s][N]
        #a_LS = all_a[s][N][iterations.index(t_1)]
        t_1_approx = - np.log(1+(1-rho) * np.linalg.norm(a1 - a_LS, ord=2) / (alpha*beta_1)) / np.log(rho+1e-8)
        # if method == "proj_subgradient": t_1_approx = 1
        all_t1_approx[s].append(t_1_approx)


        ##############################

        Delta_t_approx = np.linalg.norm(a_LS-a_star, ord=np.inf) / (alpha * beta_1)
        t_2_approx = t_1_approx + Delta_t_approx
        all_t2_approx[s].append(t_2_approx)
        #print(Delta_t)
        all_Delta_t_approx[s].append(Delta_t_approx)
        ##############################

        all_train_errors_s_N[s].append(all_train_errors[s][N][all_steps.index(t_1)])
        all_test_errors_s_N[s].append(all_errors[s][N][-1])

        ##############################

        print(t_1, t_1_approx)
        print(t_2, t_2_approx)
        print(Delta_t, Delta_t_approx)

        print("====")

In [ ]:
cols = 1
rows = len(all_s)

figsize=(8, 4)
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=False
log_y=True

color_1, color_2 = default_colors[0], default_colors[1]

p=1
for i, s in enumerate(all_s) :

    if cols == 1 :
        data_to_plot = list(zip([all_t1[s] ], [all_t1_approx[s] ], ['t_1']))
    elif cols == 2 :
        data_to_plot = list(zip([all_t1[s], all_t2[s]], [all_t1_approx[s], all_t2_approx[s]], ['t_1', 't_2']))
    elif cols == 3 :
        data_to_plot = list(zip([all_t1, all_t2, all_Delta_t], [all_t1_approx, all_t2_approx, all_Delta_t_approx], ['t_1', 't_2', '\Delta t']))

    for k, (t_experiment, t_approx, label) in enumerate(data_to_plot) :

        ax = fig.add_subplot(rows, cols, p)
        p+=1
        _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
        _, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

        ax.set_title(f"s={s}", fontsize=LABEL_FONTSIZE)

        ax.plot(all_N, t_experiment, '-', label=f'${label}$ (experiments)', color=color_1, linewidth=LINEWIDTH)
        ax.plot(all_N, t_approx, '--', label=f'$O({label})$ (theory)', color=color_1, linewidth=LINEWIDTH)

        ax.set_xlabel('N', fontsize=LABEL_FONTSIZE)
        if k==0 : ax.set_ylabel('t', fontsize=LABEL_FONTSIZE)
        ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
        ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)

        #ax.legend(fontsize=LABEL_FONTSIZE*0.8)

        if log_x : ax.set_xscale('log')
        if log_y : ax.set_yscale('log')

        ###################

        ax1.plot(all_N, all_train_errors_s_N[s], '-', label='$||X a^{(t_1)} - y^*||_2 \ \ / \ \ ||y^*||_2$', color=color_2, linewidth=LINEWIDTH)
        ax1.plot(all_N, all_test_errors_s_N[s], '--', label='$||a^{(\infty)} - a^*||_2 \ \ / \ \ ||a^*||_2$', color=color_2, linewidth=LINEWIDTH)

        #ax1.set_xlabel('N', fontsize=LABEL_FONTSIZE)
        if k==cols-1 : ax1.set_ylabel('Error', fontsize=LABEL_FONTSIZE)
        #ax1.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
        ax1.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

        #if log_x : ax1.set_xscale('log')
        if log_y : ax1.set_yscale('log')

        legend_elements = [
            Line2D([0], [0], color=color_1, linestyle='-', label=f'${label}$ (experiments)'),
            Line2D([0], [0], color=color_1, linestyle='--', label=f'$O({label})$ (theory)'),
            Line2D([0], [0], color=color_2, linestyle='-', label='$||X a^{(t_1)} - y^*||_2 \ \ / \ \ ||y^*||_2$'),
            Line2D([0], [0], color=color_2, linestyle='--', label='$||a^{(\infty)} - a^*||_2 \ \ / \ \ ||a^*||_2$')
            ]
        ax1.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)


# # Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_N_and_s_vs_times_large_{method}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

### Figure 28 (subgradient)

In [ ]:
rows, cols = 2, 2
figsize=FIGSIZE
figsize=FIGSIZE_MEDIUM
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=False
log_y=True

color_indices = np.linspace(0, 1, len(all_s)+1*0)
colors = plt.cm.viridis(color_indices)

##################### test errors
ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for i, s in enumerate(all_s) :
    #yy = [all_train_errors_s_N[s][j]  for j, N in enumerate(all_N)]
    yy = [min(all_train_errors[s][N]) for N in all_N]
    ax.plot(all_N, yy, '-', color=colors[i], label=f"s={s}", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$||X a^{(t_1)} - y^*||_2 \ \ / \ \ ||y^*||_2$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

# ##################### test errors
ax = fig.add_subplot(rows, cols, 2)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for i, s in enumerate(all_s) :
    #yy = [all_test_errors_s_N[s][j] for j, N in enumerate(all_N)]
    yy = [min(all_errors[s][N]) for N in all_N]
    ax.plot(all_N, yy, '-', color=colors[i], label=f"s={s}", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$||a^{(t_2)} - a^*||_2 \ \ / \ \ ||a^*||_2$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

# ##################### t_1
ax = fig.add_subplot(rows, cols, 3)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for i, s in enumerate(all_s) :
    yy = [all_t1[s][j] for j, N in enumerate(all_N)]
    ax.plot(all_N, yy, '-', color=colors[i], label=f"s={s}", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$t_1$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

# ##################### t_2
ax = fig.add_subplot(rows, cols, 4)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for i, s in enumerate(all_s) :
    yy = [all_t2[s][j] for j, N in enumerate(all_N)]
    ax.plot(all_N, yy, '-', color=colors[i], label=f"s={s}", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$t_2$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

# #####################

## Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

# # ##
#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_errors_vs_N_and_L_{method}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

In [ ]:
data_to_plot = list(zip([all_t1], [all_t1_approx], ['t_1']))
#data_to_plot = list(zip([all_t1, all_t2], [all_t1_approx, all_t2_approx], ['t_1', 't_2']))
#data_to_plot = list(zip([all_t1, all_t2, all_Delta_t], [all_t1_approx, all_t2_approx, all_Delta_t_approx], ['t_1', 't_2', '\Delta t']))

rows, cols = 1, len(data_to_plot)
figsize=FIGSIZE # (8, 4)
figsize=FIGSIZE_MEDIUM
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=False
log_y=True

###################################################

for k, (t_experiment, t_approx, label) in enumerate(data_to_plot) :

    ax = fig.add_subplot(rows, cols, k+1)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    #_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

    color_indices = np.linspace(0, 1, len(all_s)+1*0)
    colors = plt.cm.viridis(color_indices)

    for i, s in enumerate(all_s) :
        ax.plot(all_N, t_experiment[s], '-', label=f'${label}$ (experiments)', color=colors[i], linewidth=LINEWIDTH)
        ax.plot(all_N, t_approx[s], '--', label=f'$O({label})$ (theory)', color=colors[i], linewidth=LINEWIDTH)

    ax.set_xlabel('N', fontsize=LABEL_FONTSIZE)
    if k==0 : ax.set_ylabel('t', fontsize=LABEL_FONTSIZE)
    ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
    ########### Color bar
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_s), vmax=max(all_s)))
    sm.set_array([])  # We only need the colormap here, no actual data
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label(f'$s$', fontsize=LABEL_FONTSIZE)
    # Set the ticks to correspond to the values in `all_N`
    cbar.set_ticks(all_s)  # Sets tick positions based on `all_N`
    cbar.set_ticklabels([str(s) for s in all_s])  # Sets tick labels to match `all_N`


    if log_x : ax.set_xscale('log')
    if log_y : ax.set_yscale('log')

    legend_elements = [
        Line2D([0], [0], color='k', linestyle='-', label=f'${label}$ (experiments)'),
        Line2D([0], [0], color='k', linestyle='--', label=f'$O({label})$ (theory)')
        ]
    ax.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)


###################################################

# # Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar


##
#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_N_and_s_vs_times_small_{method}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

## Effect of $\alpha \beta$ : Figures 4 and 26

In [ ]:
n = 100 # Dimension of the signal
seed = 1
signal_distribution="normal"
#Phi = create_Fourier_basis(n) # Fourier basis
Phi = create_orthonormal_basis(n, scaler=None, seed=seed)
#Phi = np.random.randn(n, n) * (1/np.sqrt(n))
Phi = np.eye(n)

s=5
N=30
SNR=10**8 # signal to noise ratio (np.inf for no noise)
SNR=np.inf

a_star, b_star = create_signal(n, s, signal_distribution, Phi, scaler=None, seed=seed)
M, X = get_measures(N, Phi, tau=0.0, variance=1/N, seed=seed)
xi = create_noise_from_scratch(N, SNR, n, s, signal_distribution, Phi, scaler=None, seed=seed)
y_star = X @ a_star + xi

In [ ]:
#all_alpha = [1e-1]
#all_alpha = [1e-2, 1e-1]
all_alpha = [1e-2/2, 1e-2, (1/4) * (1e-2 + 1e-1), (2/4) * (1e-2 + 1e-1), (3/4) * (1e-2 + 1e-1), 1e-1]
all_alpha = sorted(all_alpha)

#all_beta_1 = [1e-5, 1e-4]
all_beta_1 = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

beta_2 = 0.0
init_scale = 1e-6

method = "subgradient"
#method = "proj_subgradient"
# method = "ISTA"

max_iter=10**8+1

all_errors = {}
all_train_errors = {}
all_iterations = {}
all_a = {}
all_a_star = {}
all_a_LS = {}

fileName = f"compressed_scaling_alpha_beta"

for i, alpha in enumerate(all_alpha):
    #print(f"alpha = {alpha}, {(i+1)}/{len(all_alpha)}")

    all_errors[alpha] = {}
    all_train_errors[alpha] = {}
    all_iterations[alpha] = {}
    all_a[alpha] = {}
    all_a_star[alpha] = {}
    all_a_LS[alpha] = {}

    for j, beta_1 in enumerate(all_beta_1) :
        print(f"alpha = {alpha}, {(i+1)}/{len(all_alpha)}, beta_1={beta_1}, {(j+1)}/{len(all_beta_1)}")

        a, errors, train_errors, all_a_tmp, iterations, a_LS = subgradient_descent(
            X, y_star, a_star, method=method, learning_rate=alpha, beta_2=beta_2, beta_1=beta_1, max_iter=max_iter, init_scale=init_scale, tol=1e-6)

        all_errors[alpha][beta_1] = errors
        all_train_errors[alpha][beta_1] = train_errors
        all_iterations[alpha][beta_1] = iterations
        all_a[alpha][beta_1] = all_a_tmp
        all_a_LS[alpha][beta_1] = a_LS


        exp_dir = f"{LOG_DIR}/{fileName}/alpha={alpha}_beta={beta_1}"
        os.makedirs(exp_dir, exist_ok=True)
        np.save(f"{exp_dir}/a_star.npy", a_star)
        np.save(f"{exp_dir}/a.npy", a)
        np.save(f"{exp_dir}/errors.npy", errors)
        np.save(f"{exp_dir}/train_errors.npy", train_errors)
        np.save(f"{exp_dir}/all_a.npy", all_a_tmp)
        np.save(f"{exp_dir}/iterations.npy", iterations)
        np.save(f"{exp_dir}/a_LS.npy", a_LS)

### Figure 26

In [ ]:
L=len(all_alpha)
cols = min(2, L)
rows = L // cols + 1 * (L % cols != 0)
figsize=(8, 4)
#figsize=(8, 6)
#figsize=(15, 10)
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=True
log_y=True

for i, alpha in enumerate(all_alpha):

    ax = fig.add_subplot(rows, cols, i+1)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    #_, ax, ax1 = get_twin_axis(ax=ax, no_twin=False)

    ax.set_title(f'$\\alpha={alpha}$', fontsize=LABEL_FONTSIZE)

    color_indices = np.linspace(0, 1, len(all_beta_1)+1*0)
    colors = plt.cm.viridis(color_indices)

    for j, beta_1 in enumerate(all_beta_1) :

        all_steps = all_iterations[alpha][beta_1]
        if log_x : all_steps = np.array(all_steps) + 1
        ax.plot(all_steps, all_errors[alpha][beta_1], '--', color=colors[j], linewidth=LINEWIDTH)
        ax.plot(all_steps, all_train_errors[alpha][beta_1], '-', label=f'$\beta$={beta_1}', color=colors[j], linewidth=LINEWIDTH)

        # Plot t_2
        t_2, t_2_index = find_stable_step_final_value(all_steps, all_errors[alpha][beta_1], K=3, tolerance_fraction=0.05, M=2)
        t = t_2
        if t is not None :
            ax.axvline(x=t, ymin=0.01, ymax=1., color=colors[j], linestyle='--', lw=1.)
            ax.plot([t, t], [0, 0], 'o', color='b')

        # B = np.array(all_b[alpha][beta_1]) # (n_steps, n)
        # y = np.linalg.norm(B, ord=1, axis=1)
        # ax1.plot(all_steps, y, "-.", color=colors[j], linewidth=1.0)

    #ax.axhline(y=0, color='gray', linestyle='--', linewidth=0.8, alpha=0.8)

    if (rows-1)*cols <= i < rows*cols : ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
    if i%cols==0 : ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE)
    ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)

    ########### Color bar

    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_beta_1), vmax=max(all_beta_1)))
    import matplotlib.colors as mcolors
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=mcolors.LogNorm(vmin=min(all_beta_1), vmax=max(all_beta_1)))

    sm.set_array([])  # We only need the colormap here, no actual data
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label(f'$\\beta$', fontsize=LABEL_FONTSIZE)
    # Set the ticks to correspond to the values in `all_beta_1`
    #cbar.set_ticks(all_beta_1)  # Sets tick positions based on `all_beta_1`
    #cbar.set_ticklabels([str(beta_1) for beta_1 in all_beta_1])  # Sets tick labels to match `all_beta_1`

    if log_x : ax.set_xscale('log')
    if log_y : ax.set_yscale('log')

    legend_elements = [
        Line2D([0], [0], color='k', linestyle='-', label=LABELS_PLOTS['train_errors']),
        Line2D([0], [0], color='k', linestyle='--', label=LABELS_PLOTS['errors']),
        ]
    ax.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

    # break

## Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

##
#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_alpha_and_beta_1_{method}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

### Figure 4

In [ ]:
# from collections import defaultdict

all_alpha_beta_1 = []
all_test_errors = []
all_t2 = []

all_test_errors_predict = []
all_t2_predict = []

for i, alpha in enumerate(all_alpha):
    for j, beta_1 in enumerate(all_beta_1) :

        all_steps = all_iterations[alpha][beta_1]
        t_2, t_2_index = find_stable_step_final_value(all_steps, all_errors[alpha][beta_1], K=3, tolerance_fraction=0.05, M=2)
        #t_1, t_1_index = find_stable_step_final_value(all_steps, all_train_errors[alpha][beta_1], K=3, tolerance_fraction=0.05, M=2)

        a_LS = all_a_LS[alpha][beta_1]
        #a_LS = all_a[alpha][beta_1][all_steps.index(t_1)]

        #
        all_alpha_beta_1.append(alpha * beta_1)

        # from relative to absolute error
        #error = all_errors[alpha][beta_1][t_2_index] * np.linalg.norm(a_star, 2)
        a_t2 = all_a[alpha][beta_1][t_2_index]
        #error =  np.abs( np.linalg.norm(a_t2, 1) -  np.linalg.norm(a_star, 1) )
        error =  np.linalg.norm(a_t2-a_star, 1) / n
        all_test_errors.append( error )

        all_test_errors_predict.append(alpha * beta_1)

        all_t2.append(t_2)


        t_2_predict = np.linalg.norm(a_LS-a_star, ord=np.inf) / (alpha * beta_1)
        all_t2_predict.append(t_2_predict)

In [ ]:
rows, cols = 1, 1
figsize=FIGSIZE
figsize=FIGSIZE_MEDIUM
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

color_1, color_2 = default_colors[0], default_colors[1]

log_x=True
log_y=True

#############################################################

ax = fig.add_subplot(rows, cols, 1)
#_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

ax.plot(all_alpha_beta_1, all_t2, "o", color=color_1, label="$t_2$", markersize=MARKERSIZE)
ss = "$\\frac{ ||\hat{a}- a^*||_{\infty} }{ \\alpha \\beta }$"
#ss = "$||\hat{b}||_{\infty} / \\alpha \\beta$"
ax.plot(all_alpha_beta_1, all_t2_predict, '*', color=color_1, label=f"{ss}", markersize=MARKERSIZE)

ax.set_xlabel('$\\alpha \\beta$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$t$', fontsize=LABEL_FONTSIZE, color=color_1)
ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)

#ax.legend(fontsize=LABEL_FONTSIZE*0.8)

if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

##################### errors

ax1.plot(all_alpha_beta_1, all_test_errors, "s", color=color_2, label="$\\frac{1}{n}||a^{(t_2)} - a^*||_1$", markersize=MARKERSIZE)
ax1.plot(all_alpha_beta_1, all_test_errors_predict, 'X', color=color_2, label=f"$\\alpha \\beta$", markersize=MARKERSIZE)


#ax1.set_xlabel('$\\alpha \\beta_1$', fontsize=LABEL_FONTSIZE)
ax1.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_2)
ax1.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
ax1.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

#if log_x : ax1.set_xscale('log')
if log_y : ax1.set_yscale('log')

#############################################################

# legend_elements = [
#     Line2D([0], [0], color=color_1, linestyle='.', label='$t_2$'),
#     Line2D([0], [0], color=color_1, linestyle='x', label=f'{ss}'),
#     Line2D([0], [0], color=color_2, linestyle='.', label='$||b^{(t_2)} - b^*||_2$'),
#     Line2D([0], [0], color=color_2, linestyle='x', label='$\\alpha \\beta_1$')
#     ]

# legend_elements = [
#     Line2D([0], [0], color=color_1, marker='o', linestyle='None', markerfacecolor=color_1, label='$t_2$', markersize=10),
#     Line2D([0], [0], color=color_1, marker='*', linestyle='None', markerfacecolor=color_1, label=f'{ss}', markersize=10),
#     Line2D([0], [0], color=color_2, marker='s', linestyle='None', markerfacecolor=color_2, label='$||b^{(t_2)} - b^*||_2$', markersize=10),
#     Line2D([0], [0], color=color_2, marker='X', linestyle='None', markerfacecolor=color_2, label='$\\alpha \\beta_1$', markersize=10),
# ]
#ax1.legend(handles=legend_elements, loc='best', fontsize=LABEL_FONTSIZE*0.6)

handles, labels = ax.get_legend_handles_labels()
handles1, labels1 = ax1.get_legend_handles_labels()
ax1.legend(handles + handles1, labels + labels1, loc='lower right', fontsize=LABEL_FONTSIZE*0.8)

## Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

##
#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_time_and_error_vs_alpha_and_beta_1_{method}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

## Scaling wrt $\tau$ (coherence) : Figures 21 and 22

In [ ]:
n = 100 # Dimension of the signal
seed = 1
signal_distribution="normal"
#Phi = create_Fourier_basis(n) # Fourier basis
Phi = create_orthonormal_basis(n, scaler=None, seed=seed)
#Phi = np.random.randn(n, n) * (1/np.sqrt(n))
Phi = np.eye(n)

s=5
a_star, b_star = create_signal(n, s, signal_distribution, Phi, scaler=None, seed=seed)

alpha = 1e-1
beta_1 = 1e-5
beta_2 = 0.0
init_scale = 1e-6

method = "subgradient"
#method = "proj_subgradient"
# method = "ISTA"

max_iter=10**7+1

all_errors = {}
all_train_errors = {}
all_iterations = {}
all_a = {}
all_a_LS = {}
all_X = {}

all_N =  [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
#all_N =  [30, 70]

all_tau = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
#all_tau = [0.0, 0.1, 0.5, 0.8, 1.0]

all_max_iter = { N : 10**7+1 for N in all_N}
all_max_iter[100] = 10**4+1

fileName = f"compressed_tau"

for i, N in enumerate(all_N):

    all_errors[N] = {}
    all_train_errors[N] = {}
    all_iterations[N] = {}
    all_a[N] = {}
    all_a_LS[N] = {}
    all_X[N] = {}

    for j, tau in enumerate(all_tau):
        print(f"N = {N}, {(i+1)}/{len(all_N)}, tau = {tau}, {(j+1)}/{len(all_tau)}")

        M, X = get_measures(N, Phi, tau=tau, variance=1/N, seed=seed+i)
        #xi = create_noise_from_scratch(N, SNR, n, s, signal_distribution, Phi, scaler=None, seed=seed+i)
        y_star = X @ a_star #+ xi


        a, errors, train_errors, all_a_tmp, iterations, a_LS = subgradient_descent(
            X, y_star, a_star, method=method, learning_rate=alpha, beta_2=beta_2, beta_1=beta_1, max_iter=max_iter, init_scale=init_scale, tol=1e-6)


        all_errors[N][tau] = errors
        all_train_errors[N][tau] = train_errors
        all_iterations[N][tau] = iterations
        all_a[N][tau] = all_a_tmp
        all_a_LS[N][tau] = a_LS
        all_X[N][tau] = X

        exp_dir = f"{LOG_DIR}/{fileName}/N={N}_beta={tau}"
        os.makedirs(exp_dir, exist_ok=True)
        np.save(f"{exp_dir}/a_star.npy", a_star)
        np.save(f"{exp_dir}/a.npy", a)
        np.save(f"{exp_dir}/errors.npy", errors)
        np.save(f"{exp_dir}/train_errors.npy", train_errors)
        np.save(f"{exp_dir}/all_a.npy", all_a_tmp)
        np.save(f"{exp_dir}/iterations.npy", iterations)
        np.save(f"{exp_dir}/a_LS.npy", a_LS)

In [ ]:
# for i, N in enumerate(all_N) :
#     for j, tau in enumerate(all_tau):
#         print(N, tau)

### Figure 21

In [ ]:
L=len(all_N)
cols = min(2, L)
rows = L // cols + 1 * (L % cols != 0)

figsize=(8, 4)
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=True
log_y=True

for i, N in enumerate(all_N) :

    ax = fig.add_subplot(rows, cols, i+1)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    #_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

    ax.set_title(f"N={N}", fontsize=LABEL_FONTSIZE)

    color_indices = np.linspace(0, 1, len(all_tau)+1*0)
    colors = plt.cm.viridis(color_indices)

    for j, tau in enumerate(all_tau):
        all_steps = all_iterations[N][tau]
        if log_x : all_steps = np.array(all_steps)+1
        T_max=2000
        ax.plot(all_steps[:T_max], all_errors[N][tau][:T_max], '--', color=colors[j], linewidth=LINEWIDTH)
        ax.plot(all_steps[:T_max], all_train_errors[N][tau][:T_max], '-', label=f'$\tau$={tau}', color=colors[j], linewidth=LINEWIDTH)

        t=None
        #t=all_t1[s][j]
        if t is not None :
            ax.axvline(x=t, ymin=0.01, ymax=1., color=colors[j], linestyle='--', lw=1.)
            ax.plot([t, t], [0, 0], 'o', color='b')

    ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
    ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE)
    ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
    ########### Color bar
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_tau), vmax=max(all_tau)))
    sm.set_array([])  # We only need the colormap here, no actual data
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label(f'$\\tau$', fontsize=LABEL_FONTSIZE)
    # Set the ticks to correspond to the values in `all_tau`
    cbar.set_ticks(all_tau)  # Sets tick positions based on `all_tau`
    cbar.set_ticklabels([str(tau) for tau in all_tau])  # Sets tick labels to match `all_tau`

    if log_x : ax.set_xscale('log')
    if log_y : ax.set_yscale('log')

    legend_elements = [
        Line2D([0], [0], color='k', linestyle='-', label=LABELS_PLOTS['train_errors']),
        Line2D([0], [0], color='k', linestyle='--', label=LABELS_PLOTS['errors']),
        ]
    ax.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

##############################################################################
##############################################################################

# # Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

##
#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_N_and_tau_{method}_n={n}_s={s}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

### Figure 22

In [ ]:
all_t1 = {}
all_t2 = {}
all_Delta_t = {}

all_train_errors_N_tau = {}
all_test_errors_N_tau = {}

for i, N in enumerate(all_N) :

    all_t1[N] = []
    all_t2[N] = []
    all_Delta_t[N] = []
    all_train_errors_N_tau[N] = []
    all_test_errors_N_tau[N] = []

    for j, tau in enumerate(all_tau):

        all_steps = all_iterations[N][tau]

        t_1, t_1_index = find_stable_step_final_value(all_steps, all_train_errors[N][tau], K=3, tolerance_fraction=0.05, M=2)
        if t_1 is None :
            t_1 = all_steps[-1]
            t_1_index = all_steps.index(t_2)

        t_2, t_2_index = find_stable_step_final_value(all_steps, all_errors[N][tau], K=3, tolerance_fraction=0.05, M=2)
        if t_2 is None :
            t_2 = all_steps[-1]
            t_2_index = all_steps.index(t_2)


        all_t1[N].append(t_1)
        all_t2[N].append(t_2)
        all_Delta_t[N].append(t_2 - t_1)
        all_train_errors_N_tau[N].append(all_train_errors[N][tau][t_1_index])
        all_test_errors_N_tau[N].append(all_errors[N][tau][-1])


In [ ]:
L=len(all_N)
cols = min(2, L)
rows = L // cols + 1 * (L % cols != 0)

figsize=FIGSIZE
figsize=FIGSIZE_MEDIUM
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

color_1, color_2 = default_colors[0], default_colors[1]

log_x=False
log_y=True

for i, N in enumerate(all_N) :

    ax = fig.add_subplot(rows, cols, i+1)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    _, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

    ax.set_title(f"N={N}", fontsize=LABEL_FONTSIZE)

    # ax.plot(all_tau, all_t1[N], "o", color=color_1, label="$t_1$", markersize=MARKERSIZE)
    # ax.plot(all_tau, all_t2[N], "*", color=color_1, label="$t_2$", markersize=MARKERSIZE)

    ax.plot(all_tau, all_t1[N], "-", color=color_1, label="$t_1$", linewidth=LINEWIDTH)
    ax.plot(all_tau, all_t2[N], "--", color=color_1, label="$t_2$", linewidth=LINEWIDTH)


    ax.set_xlabel('$\\tau$', fontsize=LABEL_FONTSIZE)
    ax.set_ylabel('$t$', fontsize=LABEL_FONTSIZE, color=color_1)
    ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
    ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)
    # Set the ticks to correspond to the values in `all_tau`
    ax.set_xticks(all_tau)  # Sets tick positions based on `all_tau`
    ax.set_xticklabels([str(tau) for tau in all_tau])  # Sets tick labels to match `all_tau`

    #ax.legend(fontsize=LABEL_FONTSIZE*0.8)

    if log_x : ax.set_xscale('log')
    if log_y : ax.set_yscale('log')

    ##################### errors


    # ax1.plot(all_L, all_train_errors_N_tau[N], 's', color=color_2, label='$||X a^{(t_1)} - y^*||_2 \ \ / \ \ ||y^*||_2$', markersize=MARKERSIZE)
    # ax1.plot(all_L, all_test_errors_N_tau[N], "X", color=color_2, label='$||a^{(t_2)} - a^*||_2 \ \ / \ \ ||a^*||_2$', markersize=MARKERSIZE)

    ax1.plot(all_tau, all_train_errors_N_tau[N], '-', color=color_2, label='$||X a^{(t_1)} - y^*||_2 \ \ / \ \ ||y^*||_2$', linewidth=LINEWIDTH)
    ax1.plot(all_tau, all_test_errors_N_tau[N], "--", color=color_2, label='$||a^{(t_2)} - a^*||_2 \ \ / \ \ ||a^*||_2$', linewidth=LINEWIDTH)

    #ax1.set_xlabel('$\\tau$', fontsize=LABEL_FONTSIZE)
    ax1.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_2)
    ax1.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
    ax1.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

    #if log_x : ax1.set_xscale('log')
    if log_y : ax1.set_yscale('log')

    #############################################################

    handles, labels = ax.get_legend_handles_labels()
    handles1, labels1 = ax1.get_legend_handles_labels()
    ax1.legend(handles + handles1, labels + labels1, loc='best', fontsize=LABEL_FONTSIZE*0.8)

    #break

## Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

##
#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_N_vs_N_tau_large_{method}_n={n}_s={s}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

**Figure 22**

In [ ]:
rows, cols = 2, 2
figsize=FIGSIZE
figsize=FIGSIZE_MEDIUM
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=False
log_y=True

color_indices = np.linspace(0, 1, len(all_tau)+1*0)
colors = plt.cm.viridis(color_indices)

##################### test errors
ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for j, tau in enumerate(all_tau) :
    #yy = [all_train_errors_N_L[N][j] for N in all_N]
    yy = [min(all_train_errors[N][tau]) for N in all_N]
    ax.plot(all_N, yy, '-', color=colors[j], label=f"$\\tau={tau}$", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$||X a^{(t_1)} - y^*||_2 \ \ / \ \ ||y^*||_2$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

##################### test errors
ax = fig.add_subplot(rows, cols, 2)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for j, tau in enumerate(all_tau) :
    #yy = [all_test_errors_N_L[N][j] for N in all_N]
    yy = [min(all_errors[N][tau]) for N in all_N]
    ax.plot(all_N, yy, '--', color=colors[j], label=f"$\\tau={tau}$", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$||a^{(t_2)} - a^*||_2 \ \ / \ \ ||a^*||_2$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

##################### t_1
ax = fig.add_subplot(rows, cols, 3)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for j, tau in enumerate(all_tau) :
    yy = [all_t1[N][j] for N in all_N]
    ax.plot(all_N, yy, '-', color=colors[j], label=f"$\\tau={tau}$", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$t_1$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

##################### t_2
ax = fig.add_subplot(rows, cols, 4)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for j, tau in enumerate(all_tau) :
    yy = [all_t2[N][j] for N in all_N]
    ax.plot(all_N, yy, '-', color=colors[j], label=f"$\\tau={tau}$", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$t_2$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

#####################

## Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

# ##
#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_N_vs_N_tau_small_{method}_n={n}_s={s}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

## Depth (L) with small init :  Figures 9, 17 and 18

In [ ]:
n = 100 # Dimension of the signal
seed = 1
signal_distribution="normal"
#Phi = create_Fourier_basis(n) # Fourier basis
Phi = create_orthonormal_basis(n, scaler=None, seed=seed)
#Phi = np.random.randn(n, n) * (1/np.sqrt(n))
Phi = np.eye(n)

s=5
#all_s =  [2]
#all_s =  [1, 5, 10, 15]
a_star, b_star = create_signal(n, s, signal_distribution, Phi, scaler=None, seed=seed)

alpha = 1e-1
beta_1 = 0.0
beta_2 = 0.0
init_scale = 1e-6

method = "subgradient"
#method = "proj_subgradient"
# method = "ISTA"

max_iter=10**6*1+1

all_errors = {}
all_train_errors = {}
all_iterations = {}
all_b = {}
all_b_LS = {}
all_X = {}


all_N =  [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
#all_N =  [30, 70]
SNR=10**8 # signal to noise ratio (np.inf for no noise)
SNR=np.inf

all_L = [1, 2, 3, 4]
# all_L = [1, 2]
# all_L = [3]

fileName = f"compressed_L_small_init"

for i, N in enumerate(all_N):
    #print(f"N = {N}, {(i+1)}/{len(all_N)}")

    M, X = get_measures(N, Phi, tau=0.0, variance=1/N, seed=seed)
    #xi = create_noise_from_scratch(N, SNR, n, s, signal_distribution, Phi, scaler=None, seed=seed)
    y_star = X @ a_star #+ xi

    all_errors[N] = {}
    all_train_errors[N] = {}
    all_iterations[N] = {}
    all_b[N] = {}
    all_b_LS[N] = {}

    all_X[N] = X

    for j, L in enumerate(all_L):
        print(f"N = {N}, {(i+1)}/{len(all_N)}, L = {L}, {(j+1)}/{len(all_L)}")

        if L==1:
            a, errors, train_errors, all_a_tmp, iterations, a_LS = subgradient_descent(
                X, y_star, a_star, method=method, learning_rate=alpha, beta_2=beta_2,
                beta_1=beta_1,
                #beta_1=1e-6,
                max_iter=max_iter, init_scale=1e-6, tol=1e-6)
        else :
            a, errors, train_errors, all_a_tmp, iterations, a_LS = deep_subgradient_descent(
                X, y_star, a_star, L=L, method=method, learning_rate=alpha, beta_2=beta_2, beta_1=beta_1, max_iter=max_iter, init_scale=1e-2, tol=1e-6)

        # b, a, errors, train_errors, all_b_tmp, iterations, b_LS = deep_subgradient_descent(
        #     X, Phi, y_star, a_star, b_star, L=L, method=method, learning_rate=alpha, beta_2=beta_2, beta_1=beta_1, max_iter=max_iter, init_scale=1e-2, tol=1e-6)

        all_errors[N][L] = errors
        all_train_errors[N][L] = train_errors
        all_iterations[N][L] = iterations
        all_b[N][L] = all_a_tmp
        all_b_LS[N][L] = a_LS


        exp_dir = f"{LOG_DIR}/{fileName}/N={N}_L={L}"
        os.makedirs(exp_dir, exist_ok=True)
        np.save(f"{exp_dir}/a_star.npy", a_star)
        np.save(f"{exp_dir}/a.npy", a)
        np.save(f"{exp_dir}/errors.npy", errors)
        np.save(f"{exp_dir}/train_errors.npy", train_errors)
        np.save(f"{exp_dir}/all_a.npy", all_a_tmp)
        np.save(f"{exp_dir}/iterations.npy", iterations)
        np.save(f"{exp_dir}/a_LS.npy", a_LS)

### Figure 17

In [ ]:
L=len(all_N)
cols = min(2, L)
rows = L // cols + 1 * (L % cols != 0)

figsize=(8, 4)
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=True
log_y=False

for i, N in enumerate(all_N) :

    ax = fig.add_subplot(rows, cols, i+1)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    #_, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

    ax.set_title(f"N={N}", fontsize=LABEL_FONTSIZE)

    color_indices = np.linspace(0, 1, len(all_L)+1*0)
    colors = plt.cm.viridis(color_indices)

    for j, L in enumerate(all_L):
        all_steps = all_iterations[N][L]
        if log_x : all_steps = np.array(all_steps) + 1
        ax.plot(all_steps, all_errors[N][L], '--', color=colors[j], linewidth=LINEWIDTH)
        ax.plot(all_steps, all_train_errors[N][L], '-', label=f'$L$={L}', color=colors[j], linewidth=LINEWIDTH)

    ax.set_xlabel('Steps (t)', fontsize=LABEL_FONTSIZE)
    ax.set_ylabel('Error', fontsize=LABEL_FONTSIZE)
    ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
    ########### Color bar
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min(all_L), vmax=max(all_L)))
    sm.set_array([])  # We only need the colormap here, no actual data
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label(f'$L$', fontsize=LABEL_FONTSIZE)
    # Set the ticks to correspond to the values in `all_L`
    cbar.set_ticks(all_L)  # Sets tick positions based on `all_L`
    cbar.set_ticklabels([str(l) for l in all_L])  # Sets tick labels to match `all_L`

    if log_x : ax.set_xscale('log')
    if log_y : ax.set_yscale('log')

    legend_elements = [
        Line2D([0], [0], color='k', linestyle='-', label=LABELS_PLOTS['train_errors']),
        Line2D([0], [0], color='k', linestyle='--', label=LABELS_PLOTS['errors']),
        ]

    ax.legend(handles=legend_elements, fontsize=LABEL_FONTSIZE*0.8)

##############################################################################
##############################################################################

# # Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

##
#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_depth_{method}_n={n}_s={s}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

### Figure 18

In [ ]:
all_t1 = {}
all_t2 = {}
all_Delta_t = {}

all_train_errors_N_L = {}
all_test_errors_N_L = {}

for i, N in enumerate(all_N) :

    all_t1[N] = []
    all_t2[N] = []
    all_Delta_t[N] = []
    all_train_errors_N_L[N] = []
    all_test_errors_N_L[N] = []

    for j, L in enumerate(all_L):

        all_steps = all_iterations[N][L]

        t_1, t_1_index = find_stable_step_final_value(all_steps, all_train_errors[N][L], K=3, tolerance_fraction=0.05, M=2)
        if t_1 is None :
            t_1 = all_steps[-1]
            t_1_index = all_steps.index(t_2)

        t_2, t_2_index = find_stable_step_final_value(all_steps, all_errors[N][L], K=3, tolerance_fraction=0.05, M=2)
        if t_2 is None :
            t_2 = all_steps[-1]
            t_2_index = all_steps.index(t_2)


        all_t1[N].append(t_1)
        all_t2[N].append(t_2)
        all_Delta_t[N].append(t_2 - t_1)
        all_train_errors_N_L[N].append(all_train_errors[N][L][t_1_index])
        all_test_errors_N_L[N].append(all_errors[N][L][-1])

In [ ]:
L=len(all_N)
cols = min(2, L)
rows = L // cols + 1 * (L % cols != 0)

figsize=FIGSIZE
figsize=FIGSIZE_MEDIUM
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

color_1, color_2 = default_colors[0], default_colors[1]

log_x=False
log_y=True

for i, N in enumerate(all_N) :

    ax = fig.add_subplot(rows, cols, i+1)
    _, ax, _ = get_twin_axis(ax=ax, no_twin=True)
    _, ax, ax1 = get_twin_axis(ax=ax, color_1=color_1, color_2=color_2, no_twin=False)

    ax.set_title(f"N={N}", fontsize=LABEL_FONTSIZE)

    # ax.plot(all_L, all_t1[N], "o", color=color_1, label="$t_1$", markersize=MARKERSIZE)
    # ax.plot(all_L, all_t2[N], "*", color=color_1, label="$t_2$", markersize=MARKERSIZE)

    ax.plot(all_L, all_t1[N], "-", color=color_1, label="$t_1$", linewidth=LINEWIDTH)
    ax.plot(all_L, all_t2[N], "--", color=color_1, label="$t_2$", linewidth=LINEWIDTH)


    ax.set_xlabel('$L$', fontsize=LABEL_FONTSIZE)
    ax.set_ylabel('$t$', fontsize=LABEL_FONTSIZE, color=color_1)
    ax.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
    ax.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_1)
    # Set the ticks to correspond to the values in `all_L`
    ax.set_xticks(all_L)  # Sets tick positions based on `all_L`
    ax.set_xticklabels([str(l) for l in all_L])  # Sets tick labels to match `all_L`

    #ax.legend(fontsize=LABEL_FONTSIZE*0.8)

    if log_x : ax.set_xscale('log')
    if log_y : ax.set_yscale('log')

    ##################### errors


    # ax1.plot(all_L, all_train_errors_N_L[N], 's', color=color_2, label='$||X a^{(t_1)} - y^*||_2 \ \ / \ \ ||y^*||_2$', markersize=MARKERSIZE)
    # ax1.plot(all_L, all_test_errors_N_L[N], "X", color=color_2, label='$||a^{(t_2)} - a^*||_2 \ \ / \ \ ||a^*||_2$', markersize=MARKERSIZE)

    ax1.plot(all_L, all_train_errors_N_L[N], '-', color=color_2, label='$||X a^{(t_1)} - y^*||_2 \ \ / \ \ ||y^*||_2$', linewidth=LINEWIDTH)
    ax1.plot(all_L, all_test_errors_N_L[N], "--", color=color_2, label='$||a^{(t_2)} - a^*||_2 \ \ / \ \ ||a^*||_2$', linewidth=LINEWIDTH)

    #ax1.set_xlabel('$L$', fontsize=LABEL_FONTSIZE)
    ax1.set_ylabel('Error', fontsize=LABEL_FONTSIZE, color=color_2)
    ax1.tick_params(axis='x', labelsize=TICK_LABEL_FONTSIZE)
    ax1.tick_params(axis='y', labelsize=TICK_LABEL_FONTSIZE, colors=color_2)

    #if log_x : ax1.set_xscale('log')
    if log_y : ax1.set_yscale('log')

    #############################################################

    handles, labels = ax.get_legend_handles_labels()
    handles1, labels1 = ax1.get_legend_handles_labels()
    ax1.legend(handles + handles1, labels + labels1, loc='best', fontsize=LABEL_FONTSIZE*0.8)

    #break

## Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

##
#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_depth_error_vs_N_and_L_large_{method}_n={n}_s={s}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

**Figure 18**

In [ ]:
rows, cols = 2, 2
figsize=FIGSIZE
figsize=FIGSIZE_MEDIUM
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=False
log_y=True

color_indices = np.linspace(0, 1, len(all_L)+1*0)
colors = plt.cm.viridis(color_indices)

##################### test errors
ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for j, L in enumerate(all_L) :
    #yy = [all_train_errors_N_L[N][j] for N in all_N]
    yy = [min(all_train_errors[N][L]) for N in all_N]
    ax.plot(all_N, yy, '-', color=colors[j], label=f"L={L}", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$||X a^{(t_1)} - y^*||_2 \ \ / \ \ ||y^*||_2$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

##################### test errors
ax = fig.add_subplot(rows, cols, 2)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for j, L in enumerate(all_L) :
    #yy = [all_test_errors_N_L[N][j] for N in all_N]
    yy = [min(all_errors[N][L]) for N in all_N]
    ax.plot(all_N, yy, '--', color=colors[j], label=f"L={L}", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$||a^{(t_2)} - a^*||_2 \ \ / \ \ ||a^*||_2$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

##################### t_1
ax = fig.add_subplot(rows, cols, 3)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for j, L in enumerate(all_L) :
    yy = [all_t1[N][j] for N in all_N]
    ax.plot(all_N, yy, '-', color=colors[j], label=f"L={L}", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$t_1$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

##################### t_2
ax = fig.add_subplot(rows, cols, 4)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for j, L in enumerate(all_L) :
    yy = [all_t2[N][j] for N in all_N]
    ax.plot(all_N, yy, '-', color=colors[j], label=f"L={L}", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$t_2$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

#####################

## Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

# ##
#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_depth_error_vs_N_and_L_small_{method}_n={n}_s={s}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

### Figure 9

In [ ]:
rows, cols = 1, 1
figsize=FIGSIZE
#figsize=FIGSIZE_MEDIUM
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)

log_x=False
log_y=False

all_L_new = [1, 2, 3, 4]
all_L_new = [1, 2, 3]

color_indices = np.linspace(0, 1, len(all_L_new)+1*0)
colors = plt.cm.viridis(color_indices)


##################### test errors
ax = fig.add_subplot(rows, cols, 1)
_, ax, _ = get_twin_axis(ax=ax, no_twin=True)
for j, L in enumerate(all_L_new) :
    #yy = [all_test_errors_N_L[N][j] for N in all_N]
    yy = [min(all_errors[N][L]) for N in all_N]
    yy = [err * np.linalg.norm(a_star, 2) for err in yy]
    ax.plot(all_N, yy, '--', color=colors[j], label=f"L={L}", linewidth=LINEWIDTH)
ax.set_xlabel('$N$', fontsize=LABEL_FONTSIZE)
ax.set_ylabel('$||a^{(t_2)} - a^*||_2$', fontsize=LABEL_FONTSIZE)
ax.tick_params(axis='both', labelsize=TICK_LABEL_FONTSIZE)
ax.legend(fontsize=LABEL_FONTSIZE*0.8)
if log_x : ax.set_xscale('log')
if log_y : ax.set_yscale('log')

#####################

## Adjust layout and add padding
fig.tight_layout(pad=2)  # Adjust padding between plots
plt.subplots_adjust(right=0.85)  # Adjust right boundary of the plot to fit color bar

# ##
#plt.savefig(f"{LOG_DIR}/compressed_sensing_scaling_depth_error_vs_N_and_L_small_{method}_n={n}_s={s}_test_error{'_logy' if log_y else ''}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()